In [1]:
import sys
import numpy as np
import pickle
sys.path.append("F:\Programming\Projects\GlassDoor sentiment analysis\Phase 1\Src")

import model
import sentiment_utils
import data_preprocessing
import feature_engineering

from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import class_weight


In [2]:
glove_path = "F:\Programming\Projects\GlassDoor sentiment analysis\Phase 1\Glove\wiki_giga_2024_100_MFT20_vectors_seed_2024_alpha_0.75_eta_0.05.050_combined.txt"
embedding_dim = 100
embeddings_index = feature_engineering.load_glove_embeddings(glove_path, embedding_dim)

Skipping bad vector at line 4928: .
Skipping bad vector at line 10309: 2
Skipping bad vector at line 11243: 1
Skipping bad vector at line 13209: 3
Skipping bad vector at line 19215: 4
Skipping bad vector at line 24251: 5
Skipping bad vector at line 24255: 1
Skipping bad vector at line 27066: 6
Skipping bad vector at line 31576: 7
Skipping bad vector at line 31684: 1
Skipping bad vector at line 31756: 1
Skipping bad vector at line 35358: 8
Skipping bad vector at line 35680: 2
Skipping bad vector at line 36266: 1
Skipping bad vector at line 38984: 1
Skipping bad vector at line 41128: 2
Skipping bad vector at line 41281: 3
Skipping bad vector at line 41659: .
Skipping bad vector at line 42177: 1
Skipping bad vector at line 44238: 9
Skipping bad vector at line 46051: 3
Skipping bad vector at line 46351: (212)
Skipping bad vector at line 47016: (888)
Skipping bad vector at line 47243: 2
Skipping bad vector at line 47331: 4
Skipping bad vector at line 48825: 2
Skipping bad vector at line 490

In [3]:
# with open(glove_path, "r", encoding="utf-8") as f:
#     for i in range(5):
#         print(f.readline())
#         print()


In [4]:
sentiment_data = sentiment_utils.load_dataframe(path='F:\Programming\Projects\GlassDoor sentiment analysis\Phase 1\Data\Sentiment_data_2.csv')
analysis_data = sentiment_utils.load_dataframe(path='F:\Programming\Projects\GlassDoor sentiment analysis\Phase 1\Data\Analysis_data.csv')
sentiment_data.head()

,headline,pros,cons,sentiment
0,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication.",Negative
1,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,Negative
2,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very...",Negative
3,Over promised under delivered,Nice staff to work with,No career progression and salary is poor,Positive
4,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",Negative


# Data Preprocessing

### Compining text data

Compining the headline, pros, cons

In [5]:
sentiment_data = feature_engineering.combine_sentiment(sentiment_data)
sentiment_data.head()

,headline,pros,cons,sentiment,text
0,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication.",Negative,"Young colleagues, poor micro management\nVery ..."
1,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,Negative,"Excellent staff, poor salary\nFriendly, helpfu..."
2,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very...",Negative,"Low salary, bad micromanagement\nEasy to get t..."
3,Over promised under delivered,Nice staff to work with,No career progression and salary is poor,Positive,Over promised under delivered\nNice staff to w...
4,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",Negative,"client reporting admin\nEasy to get the job, N..."


In [6]:
print(sentiment_data['text'][0])

Young colleagues, poor micro management
Very friendly and welcoming to new staff. Easy going ethic.
Poor salaries, poor training and communication.


### Label encoding

In [7]:
sentiment_data['sentiment_label'] = data_preprocessing.encode_label(sentiment_data, 'sentiment')
sentiment_data['sentiment_label'].head()

0    0
1    0
2    0
3    1
4    0
Name: sentiment_label, dtype: int32

In [8]:
sentiment_data['sentiment'].value_counts()

sentiment
Positive    704799
Negative    133767
Name: count, dtype: int64

### Getting a Sample from the data

In [9]:
sentiment_data_sample = sentiment_data.sample(n=10000, random_state=42)

### Splitting data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    sentiment_data_sample['text'].values, 
    sentiment_data_sample['sentiment_label'].values, 
    test_size=0.2, 
    random_state=42,
    stratify=sentiment_data_sample["sentiment_label"])


### Data Resampling

In [11]:
sentiment_data_sample['sentiment_label'].value_counts(normalize=True)

sentiment_label
1    0.8438
0    0.1562
Name: proportion, dtype: float64

In [12]:
# smote = SMOTE(random_state=42, n_jobs=-1)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### Text cleaning and Embedding 

Before training the model, the raw review data must be cleaned and transformed into a numerical format.  
The preprocessing pipeline includes the following steps:

1. **Text Cleaning**
   - Removing stop words (e.g., *the, is, in, and*).
   - Removing URLs (e.g., `http://example.com`).
   - Removing punctuations and special characters.

2. **Text Tokenization**
   - Converting text into sequences of integer tokens using a tokenizer.

3. **Embedding & Padding**
   - Mapping tokens to dense vectors (e.g., GloVe embeddings).
   - Padding sequences to a fixed length so all inputs have the same shape.


In [13]:
X_train_embedded, X_test_embedded, tokenizer = data_preprocessing.preprocess(X_train, X_test)
X_train_embedded

array([[ 139,    2,  660, ...,    0,    0,    0],
       [  10,  238,    8, ...,    0,    0,    0],
       [ 449,   84, 5430, ...,    0,    0,    0],
       ...,
       [ 131,   12,  131, ...,    0,    0,    0],
       [ 180,   44,  145, ...,    0,    0,    0],
       [ 614,  908,   40, ...,    0,    0,    0]])

### Building Embedding Matrix

In [14]:
word_index = tokenizer.word_index
num_words = min(30000, len(word_index) + 1)

embedding_matrix = feature_engineering.build_embedding_matrix(tokenizer, embeddings_index, vocab_size= num_words)
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.17394   , -0.10466   , -0.72938901, ...,  0.242697  ,
         0.060842  ,  0.216078  ],
       ...,
       [-0.089575  ,  0.26047999, -0.209249  , ...,  0.17384   ,
        -0.06661   ,  0.17456099],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.29864299, -0.52771002,  0.48807201, ..., -0.160639  ,
        -0.37101901,  0.15707099]])

# Model training

In [15]:
model_ = model.build_sentiment_model(vocab_size=num_words, 
                                    embedding_dim = embedding_dim, 
                                    embedding_matrix=embedding_matrix, 
                                    max_sequence_length=200, 
                                    lstm_units=128)
model_

Model architecture built successfully.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1504100   
                                                                 
 bidirectional (Bidirection  (None, 256)               234496    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                 

In [16]:
history = model_.fit(
    X_train_embedded, y_train,
    validation_data=(X_test_embedded, y_test),
    epochs=10,
    batch_size=64
)


Epoch 1/10
125/125 [==============================] - 142s 1s/step - loss: 0.4086 - accuracy: 0.8407 - val_loss: 0.3459 - val_accuracy: 0.8440
Epoch 2/10
125/125 [==============================] - 320s 3s/step - loss: 0.3425 - accuracy: 0.8574 - val_loss: 0.3332 - val_accuracy: 0.8580
Epoch 3/10
125/125 [==============================] - 303s 2s/step - loss: 0.3172 - accuracy: 0.8756 - val_loss: 0.3375 - val_accuracy: 0.8545
Epoch 4/10
125/125 [==============================] - 609s 5s/step - loss: 0.3026 - accuracy: 0.8814 - val_loss: 0.3361 - val_accuracy: 0.8650
Epoch 5/10
125/125 [==============================] - 532s 4s/step - loss: 0.3005 - accuracy: 0.8824 - val_loss: 0.3251 - val_accuracy: 0.8735
Epoch 6/10
125/125 [==============================] - 602s 5s/step - loss: 0.2926 - accuracy: 0.8840 - val_loss: 0.3175 - val_accuracy: 0.8670
Epoch 7/10
125/125 [==============================] - 720s 6s/step - loss: 0.2789 - accuracy: 0.8898 - val_loss: 0.3236 - val_accuracy: 0.8715

In [19]:
model_accuracy = round(float(model_.get_metrics_result()['accuracy']), 2)
model_path = f"..\\Models\\sentiment_model_{model_accuracy}.keras"
model_.save(model_path)

In [20]:
with open("..\\Models\\tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)